In [2]:
import pandas
import numpy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
#import data #python-engine to avoid probles when importing
movies=pandas.read_csv("Datasets/Boltzmann_Machines/ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1") 
ratings=pandas.read_csv("Datasets/Boltzmann_Machines/ml-1m/ratings.dat", sep="::", header=None, engine="python", encoding="latin-1") 
users=pandas.read_csv("Datasets/Boltzmann_Machines/ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1") 

In [4]:
training_set=pandas.read_csv("Datasets/Boltzmann_Machines/ml-100k/u1.base", delimiter="\t")
training_set=numpy.array(training_set, int)

In [5]:
test_set=pandas.read_csv("Datasets/Boltzmann_Machines/ml-100k/u1.test", delimiter="\t")
test_set=numpy.array(test_set, int)

In [6]:
nb_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies=int(max(max(training_set[:,1]),max(test_set[:,1])))

In [7]:
#converting the data unto an array with users in lines and movies in columns
def convert(data):
    new_data=[]
    for id_user in range(1, nb_users+1):
        id_movies=data[:,1][data[:,0]==id_user]
        id_ratings=data[:,2][data[:,0]==id_user]
        ratings=numpy.zeros(nb_movies)
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
training_set=convert(training_set)
test_set=convert(test_set)

In [12]:
training_set=torch.FloatTensor(training_set) #expect a list of lists
test_set=torch.FloatTensor(test_set)

In [13]:
#converting the ratings into binary ratings 1 (Linked) or 0 (Not Liked)
training_set[training_set==0]=-1
training_set[training_set==1]=0
training_set[training_set==2]=0
training_set[training_set>=3]=1

In [14]:
test_set[test_set==0]=-1
test_set[test_set==1]=0
test_set[test_set==2]=0
test_set[test_set>=3]=1

In [44]:
class RBM():
    def __init__(self, nv, nh): #number visible nodes, number hidden nodes
        self.W=torch.randn(nh, nv) #normal distribution 
        self.a=torch.randn(1, nh)
        self.b=torch.randn(1, nv)

    def sample_h(self, x): #x probability of h given v #x: visible nodes, inputs
        wx=torch.mm(x, self.W.t())
        activation=wx+self.a.expand_as(wx) #to be shure to add a
        p_h_given_v=torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y): #x probability of h given v #x: visible nodes, inputs
        wy=torch.mm(y, self.W)
        activation=wy+self.b.expand_as(wy) #to be shure to add a
        p_v_given_h=torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W+=(torch.mm(v0.t(), ph0)-torch.mm(vk.t(), phk)).t()
        self.b+=torch.sum((v0-vk), 0)
        self.a+=torch.sum((ph0-phk), 0)

In [52]:
nv=len(training_set[0])
nh=200
batch_size=20
rbm=RBM(nv, nh)

In [53]:
nb_epoch=30
for epoch in range(1, nb_epoch+1):
    train_lose=0
    s=0.
    for id_user in range(0, nb_users-batch_size, batch_size):
        vk=training_set[id_user: id_user+batch_size]
        v0=training_set[id_user: id_user+batch_size]
        ph0,_=rbm.sample_h(v0)
        for k in range(10): #constrive divergence
            _,hk=rbm.sample_h(vk)
            _,vk=rbm.sample_v(hk)
            vk[v0<0]=v0[v0<0]
        phk,_=rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_lose+=torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
        #train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        #train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # Average Distance here
        s+=1.
    print("epoch:  "+str(epoch)+"  lose:  "+str(train_lose/s))

epoch:  1  lose:  tensor(0.2973)
epoch:  2  lose:  tensor(0.2498)
epoch:  3  lose:  tensor(0.2526)
epoch:  4  lose:  tensor(0.2470)
epoch:  5  lose:  tensor(0.2483)
epoch:  6  lose:  tensor(0.2490)
epoch:  7  lose:  tensor(0.2472)
epoch:  8  lose:  tensor(0.2470)
epoch:  9  lose:  tensor(0.2469)
epoch:  10  lose:  tensor(0.2470)
epoch:  11  lose:  tensor(0.2474)
epoch:  12  lose:  tensor(0.2483)
epoch:  13  lose:  tensor(0.2478)
epoch:  14  lose:  tensor(0.2471)
epoch:  15  lose:  tensor(0.2463)
epoch:  16  lose:  tensor(0.2465)
epoch:  17  lose:  tensor(0.2473)
epoch:  18  lose:  tensor(0.2452)
epoch:  19  lose:  tensor(0.2465)
epoch:  20  lose:  tensor(0.2470)
epoch:  21  lose:  tensor(0.2472)
epoch:  22  lose:  tensor(0.2490)
epoch:  23  lose:  tensor(0.2483)
epoch:  24  lose:  tensor(0.2482)
epoch:  25  lose:  tensor(0.2476)
epoch:  26  lose:  tensor(0.2477)
epoch:  27  lose:  tensor(0.2474)
epoch:  28  lose:  tensor(0.2457)
epoch:  29  lose:  tensor(0.2463)
epoch:  30  lose:  tens

In [54]:
test_lose=0
s=0.
for id_user in range(0, nb_users):
    v=training_set[id_user: id_user+1]
    vt=test_set[id_user: id_user+1]
    #ph0,_=rbm.sample_h(v)
    if len(vt[vt>=0]) > 0:
        _,h=rbm.sample_h(v)
        _,v=rbm.sample_v(h)
        vk[v0<0]=v0[v0<0]
        test_lose+=torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
        #test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) # RMSE here
        #test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # Average Distance here
        s+=1.
print("  test_lose:  "+str(test_lose/s))

  test_lose:  tensor(0.2398)
